In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
from datetime import timedelta
import numpy as np
import math
import time
import pmdarima
from IPython.display import clear_output
import xgboost as xgb

In [ ]:
new_train=pd.read_csv("~/Downloads/store-sales-time-series-forecasting/new_train.csv",parse_dates=True)

new_train["date"]=pd.to_datetime(new_train["date"])

stores=pd.read_csv("~/Downloads/store-sales-time-series-forecasting/stores.csv")

new_holiday=pd.read_csv("/home/avirup/Downloads/store-sales-time-series-forecasting/new_holiday.csv",parse_dates=True)

new_holiday["date"]=pd.to_datetime(new_holiday["date"])

new_oil=pd.read_csv("~/Downloads/store-sales-time-series-forecasting/new_oil.csv")

new_oil["date"]=pd.to_datetime(new_oil["date"])

#trans_csv=pd.read_csv("/home/avirup/Downloads/store-sales-time-series-forecasting/transactions.csv")

#trans_csv["date"]=pd.to_datetime(trans_csv["date"])

#test_csv=pd.read_csv("/home/avirup/Downloads/store-sales-time-series-forecasting/test.csv")

# OIL PRICE 

In [ ]:
new_oil.set_index(new_oil["date"].values,inplace=True)

In [ ]:
new_oil.index.freq="D"

In [ ]:
new_oil.drop(columns="date",inplace=True)

# HOLIDAYS

In [ ]:
new_holiday.set_index(new_holiday["date"].values,inplace=True)

pd.DataFrame.drop(new_holiday,columns="date",inplace=True)

In [ ]:
################ creating the vectorised form of categorical variables #####################

def feature_matrix_construct(features,columns_index,df,drop=False):
    
    temp=[]
    for i in range(len(features)):
        print(str(i)+"...done")
        temp_feature_matrix=np.zeros((len(df),len(features[i])))
        for j in range(temp_feature_matrix.shape[0]):
                for k in range(len(features[i])):
                    if df.iloc[j,columns_index[i]]==features[i][k]:
                        temp_feature_matrix[j,k]=1
       # print(np.delete(temp_feature_matrix,-1,1))
        if drop==False:
            temp.append(temp_feature_matrix)
        else:
            temp.append(np.delete(temp_feature_matrix,-1,1))
        #print(temp)
    feature_matrix=np.concatenate(tuple(temp),axis=1)
    return(feature_matrix)

Y=feature_matrix_construct(features=[np.delete(new_holiday["type"].unique(),4),\
                                    np.delete(new_holiday["locale_name"].unique(),14),\
                                        np.delete(new_holiday["transferred"].unique(),0)],\
                                           columns_index=[0,1,2],df=new_holiday,drop=True)


Z=pd.DataFrame(Y,index=new_holiday.index,columns=[str(i) for i in range(Y.shape[1])])

In [ ]:
import warnings

warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)


In [ ]:
#TRAIN TEST SPLIT
split_length=15
auto_reg_order=3
oil_lag_order=4
holiday_lag_order=2
##################

############## creating lagged columns of holiday ##############################

holiday_exog=Z.copy()
iterate=holiday_exog.columns
for i in range(1,holiday_lag_order+1):
    for x in iterate:
        holiday_exog[x+" lag" +str(i)]=holiday_exog[x].shift(i)

holiday_exog.dropna(inplace=True)


############## creating lagged columns of oil ####################################

oil_exog=new_oil.copy()
for i in range(1,oil_lag_order+1):
    oil_exog["lag"+str(i)]=oil_exog["price"].shift(i)
oil_exog=oil_exog.dropna()

oil_exog.drop(columns="date",inplace=True)


############## array for appending results ####################################
error_array=[]


#################### looping over each store ##################################
#################### regression will be done ##################################
################### for each store separately #################################
for k in new_train_csv["store_nbr"].unique():
    
    ########### extracting the data for store no. k ##########################
    Y=new_train_csv[new_train_csv["store_nbr"]==k].copy()
    Y.set_index(Y["date"].values,inplace=True)
    

    ########### merging holiday and oil exogenous variables ###################
    df_exog=pd.merge(holiday_exog,oil_exog,how="inner",left_index=True,right_index=True)
    
    
    idx=pd.date_range(df_exog.index[0],pd.to_datetime("2017-07-31"),freq="D")
    
    ########### creating dataframes for sales and promotions #################
    df_sales=pd.DataFrame(columns=new_train_csv["family"].unique(),index=idx)
    df_promo=pd.DataFrame(columns=new_train_csv["family"].unique(),index=idx)
    
    


    
    for x in df_sales.index:
        df_sales.loc[x,:]=np.log(Y.loc[x]["sales"].values+1)
        df_promo.loc[x,:]=np.log(Y.loc[x]["onpromotion"].values+1)
        
        
        
    ########### merging exogenous variables with promotion #################
            

    df_ind=pd.merge(df_promo,df_exog,how="inner",left_index=True,right_index=True)
    

    
    ########### adding lagged sales to exogenous variables for final independent variables ################
    
    iterate=df_sales.columns
    for i in range(0,auto_reg_order):
        for x in iterate:
            df_ind[x+" lag" +str(i+1)]=df_sales[x].shift(i+1)
            
    df_ind.dropna(inplace=True)
            
    ########## creating an object of xgb regressor ############

    reg_model=xgb.XGBRegressor()

    ########## fitting/ training the regressor ###################

    reg_model.fit(df_ind.values.astype(np.float32)[:,:],df_sales.values.astype(np.float32)[auto_reg_order:,:])
    
    ########## predicted output over the train set ###############

    pred_train=pd.DataFrame(reg_model.predict(df_ind.values.astype(np.float32)[:,:]),columns=df_sales.columns)


    
    ########### creating the test set for a 15 day split from 2017-08-1 to 2017-08-15 ###################
    df_promo_test=pd.DataFrame(columns=new_train_csv["family"].unique(),index=Y["date"].unique()[-split_length:])
    df_sales_test=pd.DataFrame(columns=new_train_csv["family"].unique(),index=Y["date"].unique()[-split_length:])


    for x in df_sales_test.index:

        df_sales_test.loc[x,:]=Y.loc[x]["sales"].values
        df_promo_test.loc[x,:]=Y.loc[x]["onpromotion"].values          
            
    
    
    
    ############ defining the exogenous variables for test ##############################
    
    df_exog_test=pd.merge(holiday_exog,oil_exog,how="inner",left_index=True,right_index=True)

    df_ind_test=pd.merge(df_promo_test,df_exog,how="inner",left_index=True,right_index=True)

    
    
    ############ adding the lagged sales to exogenous, uses the already predicted values ##########
    ######## the prediction for 15 day prediction is built from 1 day prediction at a time ########
    
    
    temp_df_sales=df_sales.copy()
    
    
    pred_test_array=[]
    for i in range(split_length):
        
        ########## exogenous variable for one step ahead prediction #####################
        df_ind_test_single=df_ind_test.iloc[i:i+1,:].copy()
        
        ########## adding the lagged sales to exogenous variable #################
        
        iterate=temp_df_sales.columns
        for j in range(0,auto_reg_order):
            for x in iterate:
                df_ind_test_single[x+" lag" +str(j+1)]=temp_df_sales[x].shift(j)[-1]
        
        ######## one step ahead prediction #########################
        
        pred_test_array.append(reg_model.predict(df_ind_test_single.values.astype(np.float32))[-1])
        
        ######## adding the prediction to existing sales for lagged sales calculation ###########
        
        temp_df_sales.loc[temp_df_sales.index[-1]+timedelta(days=1)]=pred_test_array[-1]
        
        
    ##### prediction before appling ARIMA or MA #####################
    
    pred_test=pd.DataFrame(pred_test_array,columns=df_sales.columns) 
    
    ##### creating data frame for post ARIMA prediction #############
    
    pred_arima_test=pd.DataFrame(pred_test_array,columns=df_sales.columns)
    
    ##### residue from regression #######
    
    residue=pd.DataFrame(pred_train.values-df_sales.values[auto_reg_order:],columns=train_csv["family"].unique(),index=df_ind.index)

    ##### Applying MA to each column of residue separately ############
    
    for x in residue.columns:
        print(("Analysing store: {} and family: {}").format(k,x))

        arima_model=pmdarima.auto_arima(residue[x],trace=True,suppress_warnings=True,max_p=0,start_p=0)
        arima_model_forecast=arima_model.predict(n_periods=15)

        
        ########## adding prediction to prection from regression #######
        pred_arima_test[x]=pred_test[x].values-arima_model_forecast.values

        
    ########## replacing negative values with zeros in prediction dataframes ############
    for z in pred_test.index:
        for y in pred_test.columns:

            if pred_test.loc[z,y]<=0:
                pred_test.loc[z,y]=0
                
            if pred_arima_test.loc[z,y]<=0:
                pred_arima_test.loc[z,y]=0
    
    
    ############ LOG MSE for each store #################
    error=math.sqrt(np.sum(np.square(np.log(pred_test.iloc[:,:].values.astype(np.float32)+1)-np.log(df_sales_test.iloc[:,:].values.astype(np.float32)+1)))/(15*33))
    error_array.append([error,df_sales_test,pred_test,pred_arima_test])
    clear_output(wait=True)
    print(("error for {}: {}").format(k,error))

In [ ]:
############## comparing sales test and predicted plots ################
############################ for 5 stores ##############################
for k in range(0,5):
    plt.figure(figsize=(20,50))
    plot_data=error_array[k]
    for j in range(len(plot_data[1].columns)):
        plt.subplot(11,3,j+1)
        plt.plot([i for i in range(split_length)],plot_data[1][plot_data[1].columns[j]],label="sales")
        plt.plot([i for i in range(split_length)],plot_data[2][plot_data[1].columns[j]],label="pred_pre_arima")
        plt.plot([i for i in range(split_length)],plot_data[3][plot_data[1].columns[j]],label="pred_post_arima")
        plt.legend()
        plt.title(plot_data[1].columns[j])
    plt.suptitle(("store number {}").format(k+1))